In [189]:
# The code was removed by Watson Studio for sharing.

In [190]:
import json
import pandas as pd

#### Takes output from Watson Service Transformer with speaker_labels = True

In [191]:
stt = pd.read_csv(project.get_file('SampleTranscriptionsSpeakerD_Workflow.csv'), sep='|')

In [192]:
pd.set_option('max_colwidth',10000)
stt.head(1)

audio_file  \
0  incomingcall_sample4 copy 2.mp3   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

#### Extracting speaker labels and talk time from data frame in parts

In [193]:
for index,row in stt.iterrows():
    audiofile = row['audio_file'] #saves the audio_file to reference as key later 
    stt_json_str_data = json.loads(row['stt_response']) #reads json object in stt_response column 
            
    # puts the whole speaker_label json field in a dataframe, giving columns for: speaker, to, from, confidence, final - see json structure for info 
    labels = pd.DataFrame.from_records(stt_json_str_data['speaker_labels']) 
            
    # time stamp for each word in transcription alternative  -returns data frame with a column containing the word, the 'to', and 'from' of utterance 
    transcript_tstamps = pd.DataFrame.from_records([t for r in stt_json_str_data['results'] for a in r['alternatives'] for t in a['timestamps']], columns=['word', 'from', 'to'] )
            
    #merging above two dataframes using 'to' and 'from' as keys
    df = labels.merge(transcript_tstamps) 
    df = df.sort_values(['from', 'to'])
            
    #aggregating sripts by speaker 
    df['current_speaker'] = (df.speaker.shift() != df.speaker).cumsum()
    transcripts = df.groupby('current_speaker').agg({'word': lambda x: ' '.join(x),'speaker': min}).rename(columns={'word': 'transcript'})
    transcripts[['speaker', 'transcript']].to_json(orient='records')
    transcripts = df.groupby('current_speaker').agg({'word': lambda x: ' '.join(x),'speaker': min,'from': min,'to': max}).rename(columns={'word': 'transcript'})
            

In [194]:
# puts the whole speaker_label json field in a dataframe, giving columns for: speaker, to, from, confidence, final - see json structure for info 
labels.head(3)

,confidence,final,from,speaker,to
0,0.50,False,19.49,0,19.75
1,0.67,False,20.97,1,21.28
2,0.67,False,21.28,1,21.55


In [195]:
# time stamp for each word in transcription alternative  -returns data frame with a column containing the word, the 'to', and 'from' of utterance 
transcript_tstamps.head(3)

,word,from,to
0,hi,19.49,19.75
1,hello,20.97,21.28
2,hi,21.28,21.55


In [196]:
#merging above two dataframes using 'to' and 'from' as keys
df = labels.merge(transcript_tstamps) 
df = df.sort_values(['from', 'to'])
df.head(3)

,confidence,final,from,speaker,to,word
0,0.50,False,19.49,0,19.75,hi
1,0.67,False,20.97,1,21.28,hello
2,0.67,False,21.28,1,21.55,hi


In [197]:
#aggregating sripts by speaker 
df['current_speaker'] = (df.speaker.shift() != df.speaker).cumsum()
transcripts = df.groupby('current_speaker').agg({'word': lambda x: ' '.join(x),'speaker': min}).rename(columns={'word': 'transcript'})
transcripts[['speaker', 'transcript']].to_json(orient='records')
transcripts = df.groupby('current_speaker').agg({'word': lambda x: ' '.join(x),'speaker': min,'from': min,'to': max}).rename(columns={'word': 'transcript'})
transcripts.head()        

,transcript,speaker,from,to
current_speaker,,,,
1,hi,0,19.49,19.75
2,hello hi I'm looking for Michels,1,20.97,22.91
3,and this,0,24.22,24.78
4,hi Michelle my name is tax and I'm calling on behalf of clear that dot com I saw your attitude Craig's list and you mentioned you provide services perfect sitting correct it has H. yeah well the reason why I'm calling is because we have a network of customers in Vancouver who are interested in having their services done so we saw your ad and we know your business can help out that customers looking for a dog in house sitters would you like to connect with our customers we have in your area this is going to be at no cost to you Michelle,1,25.66,53.64
5,I'm sure,0,55.46,56.35


### Full function
### Putting it all together - FlattenedSTT() ingests output DataFrame from Transcription with Speaker Diarization and iterates through JSON object in every row and manipulates it to extract speaker labels and talk time 
#### Returns a dataframe with original columns (transcript, JSON output, audio_file, and adds a 'flattened transcription' in the format of 'speaker 0: hello, speaker 2: how are you?' and call talk time (do not confuse with call/recording length)

In [198]:
def FlattenedSTT(stt_df):
    #creating empty dataframe to place all results
    finalflattened=pd.DataFrame(columns=['audio_file','flattened_transcript','talk_time'])
    for index,row in stt_df.iterrows():
        audiofile = row['audio_file'] #saves the audio_file to reference as key later 
        try: 
            stt_json_str_data = json.loads(row['stt_response']) #reads json object in stt_response column 
            
            # puts the whole speaker_label json field in a dataframe, giving columns for: speaker, to, from, confidence, final - see json structure for info 
            labels = pd.DataFrame.from_records(stt_json_str_data['speaker_labels']) 
            
            # time stamp for each word in transcription alternative  -returns data frame with a column containing the word, the 'to', and 'from' of utterance 
            transcript_tstamps = pd.DataFrame.from_records([t for r in stt_json_str_data['results'] for a in r['alternatives'] for t in a['timestamps']], columns=['word', 'from', 'to'] )
            
            #merging above two dataframes using 'to' and 'from' as keys
            df = labels.merge(transcript_tstamps) 
            df = df.sort_values(['from', 'to'])
            
            #aggregating sripts by speaker 
            df['current_speaker'] = (df.speaker.shift() != df.speaker).cumsum()
            transcripts = df.groupby('current_speaker').agg({'word': lambda x: ' '.join(x),'speaker': min}).rename(columns={'word': 'transcript'})
            transcripts[['speaker', 'transcript']].to_json(orient='records')
            transcripts = df.groupby('current_speaker').agg({'word': lambda x: ' '.join(x),'speaker': min,'from': min,'to': max}).rename(columns={'word': 'transcript'})
            
            #starting empty objects to use in below loop 
            flattened=''
            duration=0 
            
            #iterate through the generated dataframe that contains speaker/word/to/from to generate a transcript 
            for index,row in transcripts.iterrows():
                
                flattenedmore = 'speaker {}: '.format(row['speaker'])+'{} '.format(row['transcript'])
                flattened += flattenedmore
                uttduration = row['to']-row['from']
                duration += uttduration     
            finalflattened = finalflattened.append({'audio_file':audiofile,'flattened_transcript':flattened,'talk_time':duration},ignore_index=True)
            finalflattened['flattened_transcript']=finalflattened['flattened_transcript'].str.replace('%HESITATION','')
        
        except:
            pass
    
    finalflattened = finalflattened.append({'audio_file':audiofile,'flattened_transcript':flattened,'talk_time':duration},ignore_index=True)
    return finalflattened

In [199]:
flat_stt = FlattenedCSVfile(stt)

In [200]:
stt2 = stt.merge(flat_stt,on='audio_file',how='inner')

In [201]:
stt2.head(2)

audio_file  \
0  incomingcall_sample4 copy 2.mp3   
1         outboundcall_sample3.mp3   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [202]:
project.save_data('SampleTranscriptions_SpeakerLabels.csv', stt2.to_csv(index=False, sep='|'), overwrite=True)

{'file_name': 'SampleTranscriptions_SpeakerLabels.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'audioworkflow-donotdelete-pr-yrglph8ci3mfxo',
 'asset_id': '4c311992-2953-47e9-bfd3-c2e54cb5955f'}

### TransformSTT() 
#### For manipulation purposes- function ingests DataFrame from above function converts takes the dataframe i had generated with the 'flattened script' and converts each full transcript into broken down utterances. 
#### it is assuming speaker 0 is the agent and the other is caller. 
#### For example, it takes a column with a transcript "speaker 0: hi how are you speaker 1: goood, you?"
#### The output would have "hi how are you" as the first row "Agent" and "good you" as the first row "caller"
##### will need to group by audio_file to make usable in reports - helps visualize exchanges

In [206]:
def TransformSTT(sttdf):
    df=pd.DataFrame(columns=['Agent','Caller Response','audio_file', 'stt_response','transcript', 'flattened_transcript', 'talk_time'])

    for index,row in sttdf.iterrows():
        a=row['audio_file']
        b=row['stt_response']
        c=row['transcript']
        d=row['flattened_transcript']
        e=row['talk_time']
        
        stt = row['flattened_transcript'].split('speaker ') #splits the transcript in pieces broken down by 'speaker' leaving the speaker number 
        try:
            stt.remove('')
        except:
            pass
        
        start=stt[0].split(':')[0] #this marks the starting caller to be the first number found in the transcript "0: hello" - speaker start would be 0 
        
        for i in stt:
    
            try:
                speakerno,utterance=i.split(':')  #speaker number in transcript and utterance said by speaker
                speakerutt=utterance
                
                #checking if the speaker number in current utterance is the first speaker in the overall conversation 
                if speakerno == start:
                    #if the speaker is the first, we are assuming it is the call center agent and assigning it under the agent column 
                    #had to add an index column to use below
                    df=df.append({'Agent':speakerutt,'audio_file':a,'stt_response':b,'transcript':c,'flattened_transcript':d,'talk_time':e,'idx':a},ignore_index=True)
                    df.set_index('idx',inplace=True)
                else:
                    #if the speaker isnt the first speaker on the call, we assign it under the caller agent 
                    #since python inherently tries to put the next speaker utternace under the next column, im forcing it to look for the previous index in row 
                    df.loc[a,['Caller Response']] = speakerutt
                    
            except:
                pass
    df.reset_index(drop=True,inplace=True)
    df.drop(columns=['stt_response'],inplace=True) #dropping might not use 
    return df

In [207]:
stt3 = TransformSTT(stt2)

In [208]:
stt3.head(40)

,Agent,Caller Response,audio_file,transcript,flattened_transcript,talk_time
0,your listening to Mattel Solutions podcast for more information about my job log on to W. W. W. dot John Sessoms dot com in the address and phone call recording four zero six five three nine one two zero two,four zero six five three nine one two zero two is that correct,incomingcall_sample4 copy 2.mp3,your listening to Mattel Solutions podcast for more information about my job. log on to W. W. W. dot John Sessoms dot com in the address and phone call recording. four zero six five three nine one two zero two. four zero six five three nine one two zero two is that correct. yes how may I help you today. log off zero seven I can get the blackbird Corp. Candela Warrant Martin have S. thank you very proceed for security reasons of your account can you please verify some information for me first. yep can have the first and last name of the account holder. your million. K.. Angelo my C. Hewitt today finance. R. I. canister Al Milligan can you please verify the billing zip code. five nine seven one four. and also the last four digits of your social security number. five four three thanks very much for that information just to make Platonist Centrale Yusti V. letter regarding his locker torch and you want to take advantage of this offer. yes ok Martin have this issue that I V. filling out all the information needed for H. change and will be processed within twenty four hours from now and I just want to let you know that the new device arrive within three to five business days. please do not return call Paul has received the new device and I just want to let you know that if there is a telephone snobs return within thirty days you will be subject to a hundred fifty dollar charge builder account. also because forget to take the sim card out of the original phone in places in the new phone okay. and also you will be receiving a customer service R. meted out Hines ATP's service by eighty features by email U. S. mail this R. meal including activation fee however this is not a price you and you will not be charged Peter Polyphase for any confusion this may cost and if you have any further questions regarding this please feel free to six one one from eighteen hundred okay. and that is sending a return label itself for you to return the device and just thank you very five or they will need to send a return label you want me to send it by email Archer shipping address. sure. okay so I'm just going to verify that would be yes ninety eight masters drive elevate. One Tanner. five nine seven one four is that correct. yes okay and is this the same address from going to ship the blackberry torch. yes ok and also once you receive the block return labels just simply S. sixty phones off record in any U. S. B. Springboks okay. any U. S. P. S. blue box. ok okay so I'm Johnson that all information needed here for the exchange do you have any questions regarding information that I provided you Mister Milligan. no is there anything else I can help you with. okay we do value presheaf services thank you for calling eighteen team have a great day. thank you okay bye bye. thank you for listening. find out more about our services by visiting. Magellan dash Lucien's dot com or call our toll free number. K. escrow.,speaker 1: your listening to Mattel Solutions podcast for more information about my job log on to W. W. W. dot John Sessoms dot com in the address and phone call recording four zero six five three nine one two zero two speaker 2: four zero six five three nine one two zero two is that correct speaker 1: yes speaker 2: how may I help you today speaker 1: log off zero seven I can get the blackbird Corp speaker 2: Candela Warrant Martin have S. thank you very proceed for security reasons of your account can you please verify some information for me first speaker 1: yep speaker 2: can have the first and last name of the account holder speaker 1: your million speaker 2: K. Angelo my C

In [209]:
project.save_data('SampleTranscriptions_UtteranceBreakdown.csv', stt3.to_csv(index=False, sep=','), overwrite=True)

{'file_name': 'SampleTranscriptions_UtteranceBreakdown.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'audioworkflow-donotdelete-pr-yrglph8ci3mfxo',
 'asset_id': '4f7928fa-6ea6-4b39-923a-aa152b242290'}

#### Function iterating through the above output and outputting a JSON file that can be imported into other systems for further manipulation 

In [294]:
import numpy as np

def GenerateJSON_v1(df):
    
    df = df.replace(np.nan, '', regex=True)
    tmpall_grouped=df.groupby('audio_file')
    
    for group_name, group in tmpall_grouped:
    
        
        #print('Group {}'.format(group_name))
        
        meta = group[['audio_file','talk_time']]
        meta.drop_duplicates(keep='first',inplace=True)
        metadic=meta.to_dict('records')
        
        transcript=group[['transcript']] 
        transcript.drop_duplicates(keep='first',inplace=True)
        
        transcriptdic=transcript.to_dict(orient='records')
        
        results =group[['Agent','Caller Response']]
        results.reset_index(inplace=True,drop=True)
        resultsdic = results.to_dict(orient='index')
        
        discoverydic= {'data':metadic,'responses':resultsdic,'transcript':transcriptdic}
    
        resultsjson = json.dumps(discoverydic,indent=4)
        
        obj = open("V1 {}.json".format(group_name), "w+")
        project.save_data(file_name="V1 {}.json".format(group_name),data=obj.read(), set_project_asset=True, overwrite=True)
        obj.close()
    
        #print(resultsjson)
        #print('******')
        #print('******')

In [295]:
GenerateJSON_v1(stt3)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


{
    "data": [
        {
            "audio_file": "incomingcall_sample1.mp3",
            "talk_time": 495.75000000000017
        }
    ],
    "responses": {
        "0": {
            "Agent": " you have reached as and stuff like that this is Donna I'll be assisting you with your increase today please be informed that this call is being recorded and monitored for quality assurance purposes how may I help you ",
            "Caller Response": "  I World I bought a right I got this  essence of Oregon oil  for shipping handling and handling Costa Friday nine nine sample over and  if I want to cancel the order I had to do it within fifteen H. and that is  when I want wanted to do I didn't want to I didn't want to get you know like a monthly sure what is it eighty three dollars a month  K. for that "
        },
        "1": {
            "Agent": " okay I'm more than happy happy to assist you  for me to be able to pull up your  subscription his pager me your first and your last name ",
 